In [ ]:
import os
os.chdir(r"Q:/sachuriga/Sachuriga_Python/quattrocolo-nwb4fp/src")


import pandas as pd
from neurochat.nc_data import NData
from neurochat.nc_spike import NSpike
from neurochat.nc_spatial import NSpatial
import neurochat.nc_plot as nc_plot
from neurochat.nc_lfp import NLfp
import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import numpy as np
import math
import pynapple as nap
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize

import sys
import nwb4fp.analyses.maps as mapp
from nwb4fp.analyses.examples.tracking_plot import plot_ratemap_ax,plot_path
from nwb4fp.analyses.fields import separate_fields_by_laplace, separate_fields_by_dilation,find_peaks,separate_fields_by_laplace_of_gaussian,calculate_field_centers,distance_to_edge_function, remove_fields_by_area, map_pass_to_unit_circle,which_field,compute_crossings
from elephant.statistics import time_histogram, instantaneous_rate
from nwb4fp.analyses import maps
from nwb4fp.analyses.data import pos2speed,speed_filtered_spikes,load_speed_fromNWB,load_units_fromNWB,get_filed_num,unit_location_ch,calculate_spatial_coherence,calculate_spatial_stability,coherence, find_run_indices
from scipy.ndimage import gaussian_filter
import ast
import pandas as pd
# df = pd.read_csv(rf'{ephys_path}\{animal}\{animal}_{day}_unitmatchResults\MatchTable.csv')
# df = pd.read_csv(r'S:\Sachuriga\Ephys_Recording\CR_CA1/65165/65165_2023-07-10_unitmatchResults/MatchTable.csv')


In [ ]:
pairs_df

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp, mannwhitneyu
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(7.2, 3.7),dpi=1200)
plt.rcParams.update({'font.size': 8})
gs = gridspec.GridSpec(2, 4, height_ratios=[1, 1], width_ratios=[0.9, 0.9, 0.9, 0.9])  # First row taller
plt.rcParams.update({
    'axes.labelpad': -0.1,
    'ytick.major.pad': -0.1,
    'xtick.major.pad': -0.1,
    'ytick.major.size': 2,
    'xtick.major.size': 2
})



ax1_2 = fig.add_subplot(gs[0, 1])
ax1_3 = fig.add_subplot(gs[0, 2])
ax1_4 = fig.add_subplot(gs[0, 3])

ax2_1 = fig.add_subplot(gs[1, 0], projection='3d')
ax2_2 = fig.add_subplot(gs[1, 1])
ax2_3 = fig.add_subplot(gs[1, 2])
ax2_4 = fig.add_subplot(gs[1, 3])


pairs_df=pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/Remapping_ratemaps_unsmoothed.pkl")
# Assuming your data is loaded in pairs_df
df = pairs_df[(pairs_df['cell_type1']=="pyramidal") 
              & (pairs_df['cell_type2']=="pyramidal")]


# df = pairs_df[(pairs_df['neuron location 1']=="superficial") 
#               & (pairs_df['neuron location 2']=="superficial")]


# Define control and experimental animal_ids
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']

# Split into three groups based on Session1 and Session2
ab_group = df[(df['Session1'] == 1.0) & (df['Session2'] == 2.0)]  # AB group
ba_group = df[(df['Session1'] == 2.0) & (df['Session2'] == 3.0)]  # BA' group
aa_group = df[(df['Session1'] == 1.0) & (df['Session2'] == 3.0)]  # AA' group

# Split into control and experimental groups
control_ab = ab_group[ab_group['animal_id'].isin(control_ids)]
control_ba = ba_group[ba_group['animal_id'].isin(control_ids)]
control_aa = aa_group[aa_group['animal_id'].isin(control_ids)]

exp_ab = ab_group[ab_group['animal_id'].isin(exp_ids)]
exp_ba = ba_group[ba_group['animal_id'].isin(exp_ids)]
exp_aa = aa_group[aa_group['animal_id'].isin(exp_ids)]

# Prepare plotting data
plot_data = []

# Add control group data
for group_num, group_df in enumerate([control_ab, control_ba, control_aa], 1):
    for corr_value in group_df['Corr']:
        plot_data.append({
            'Corr': corr_value,
            'Group': f'Group{group_num}',
            'Condition': 'Control'
        })

# Add experimental group data
for group_num, group_df in enumerate([exp_ab, exp_ba, exp_aa], 1):
    for corr_value in group_df['Corr']:
        plot_data.append({
            'Corr': corr_value,
            'Group': f'Group{group_num}',
            'Condition': 'Experimental'
        })

# Create DataFrame for plotting
plot_df = pd.DataFrame(plot_data)

axes_cdf = [ax1_2,ax1_3,ax1_4]

group_labels = ['S1=1.0, S2=2.0', 'S1=2.0, S2=3.0', 'S1=1.0, S2=3.0']
alternatives = ['two-sided', 'two-sided', 'two-sided']  # AB: Exp > Control, BA': two-sided, AA': Exp < Control

# Perform statistical tests and create plots
for i, group in enumerate(['Group1', 'Group2', 'Group3']):
    # Get data for current group
    control_data = plot_df[(plot_df['Group'] == group) & (plot_df['Condition'] == 'Control')]['Corr']
    exp_data = plot_df[(plot_df['Group'] == group) & (plot_df['Condition'] == 'Experimental')]['Corr']
    
    # Perform KS test
    ks_statistic, ks_p_value = ks_2samp(control_data, exp_data, alternative=alternatives[i])
    # Perform Mann-Whitney U test
    mw_statistic, mw_p_value = mannwhitneyu(control_data, exp_data, alternative=alternatives[i])
    print(f"Group{i+1} ({group_labels[i]}): KS p-value = {ks_p_value:.4f}, MW p-value = {mw_p_value:.4f}")
    # Plot CDF (second row)
    sns.ecdfplot(data=control_data, ax=axes_cdf[i], color='blue', label='Control')
    sns.ecdfplot(data=exp_data, ax=axes_cdf[i], color='red', label='Experimental')
    

    # Customize CDF subplot
    #axes_cdf[i].set_title(f'Mann-Whitney p = {mw_p_value:.3f}', fontsize=10)
    axes_cdf[i].set_xlabel('Spatial correlation')
    axes_cdf[i].set_ylabel('% Neuron')
    axes_cdf[i].legend()
    axes_cdf[i].grid(False)
    axes_cdf[i].set_aspect('auto')
    axes_cdf[i].spines['right'].set_visible(False)
    axes_cdf[i].spines['top'].set_visible(False)
    axes_cdf[i].legend().set_visible(False)
    axes_cdf[i].set_xlim([-0.4, 1])

        # Calculate means and SEM
    control_mean = np.mean(control_data)
    exp_mean = np.mean(exp_data)
    control_sem = stats.sem(control_data)
    exp_sem = stats.sem(exp_data)

    # Create inset axes for bar plot in top-right corner
    inset_ax = inset_axes(axes_cdf[i], width="80%", height="80%", loc='lower right',
                     bbox_to_anchor=(0.7, 0, 0.3, 0.3),  # 4-tuple: x0, y0, width, height
                     bbox_transform=axes_cdf[i].transAxes)
    
    inset_ax.bar([0.2, 0.8], [control_mean, exp_mean], yerr=[control_sem, exp_sem], 
                 color=['blue', 'red'], alpha=0.9, width=0.45, capsize=2)
    inset_ax.set_xticks([])
    inset_ax.set_yticks([])
    #inset_ax.set_xticklabels(['CRs +', 'CRs -'], fontsize=8)
    #inset_ax.set_ylabel('Mean', fontsize=8)
    inset_ax.tick_params(axis='both', labelsize=6)
    inset_ax.spines['top'].set_visible(False)
    inset_ax.spines['right'].set_visible(False)
    inset_ax.spines['left'].set_visible(False)
    inset_ax.set_ylim([0, 0.6])


all_rates = np.load(r"Q:\sachuriga\CR_CA1_paper\tables/population_vectors.npy")
rate_maps = np.array(all_rates)
rate_maps=rate_maps[1:8,:,:]
n_cells, height, width = rate_maps.shape
position=(18, 5)
x_pos, y_pos = position
ax= ax2_1

# Plot each rate map as a 2D heatmap at different Z levels
for z in range(n_cells + 2):

    if z < n_cells:
        # Create a grid for the heatmap
        X, Y = np.meshgrid(np.arange(width), np.arange(height))
        # Normalize the rate map for colormap
        rate_map = rate_maps[z]
        # Create an RGBA image for the heatmap
        cmap = plt.cm.hot(rate_map / np.max(rate_map))
        # Plot the heatmap as a collection of colored squares
        for i in range(height):
            for j in range(width):
                color = cmap[i, j]
                ax.bar3d(j, i, z, 1, 1, 0, color=color, shade=False, alpha=0.6)

    if z == n_cells+1:
        X, Y = np.meshgrid(np.arange(width), np.arange(height))
        # Normalize the rate map for colormap
        rate_map = rate_maps[6]
        # Create an RGBA image for the heatmap
        cmap = plt.cm.hot(rate_map / np.max(rate_map))
        # Plot the heatmap as a collection of colored squares
        for i in range(height):
            for j in range(width):
                color = cmap[i, j]
                ax.bar3d(j, i, -3, 1, 1, 0, color=color, shade=False, alpha=0.6)

# Add a vertical line at (x=15, y=15)
ax.plot3D([x_pos, x_pos], [y_pos, y_pos],[-4, 8], color='blue')

# Set labels and limits
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Cell Index')
ax.set_zlim(-2, n_cells-1)
ax.set_xlim(0, width)
ax.set_ylim(0, height)

# Customize z-axis ticks to show "1, 2, 3, ..., n"
z_ticks = np.arange(n_cells)
z_labels = [str(i+1) for i in range(n_cells)]
z_labels[-1] = 'n'  # Replace the last label with 'n'
ax.set_zticks(z_ticks)
ax.set_zticklabels(z_labels)

# Adjust the viewing angle for a more parallel perspective
ax.view_init(elev=10, azim=45)  # Elevation = 10°, Azimuth = 45°
ax.axis('off')
ax.grid('off')

# Assume processed_df_normalized and population_vector_correlation are defined
patterns = [("A", "B"), ("B", "C"), ("A", "C")]

# Set up figure with subplots (2 rows, 3 columns)
axes = [ax2_2,ax2_3,ax2_4]

for idx, (s1, s2) in enumerate(patterns):
    # Filter rows where both s1 and s2 have values (not null)
    filtered_df = processed_df_normalized.dropna(subset=[s1, s2])
    control_df = filtered_df[filtered_df['group'] == 'control']
    exp_df = filtered_df[filtered_df['group'] == 'exp']

    def to_3d_array(df, column, i=2, j=2):
        values = df[column].values
        n = len(values)
        return np.array([np.full((i, j), val) for val in values])

    i, j = 20, 20
    control_a_3d = to_3d_array(control_df, s1, i, j)
    control_b_3d = to_3d_array(control_df, s2, i, j)
    exp_a_3d = to_3d_array(exp_df, s1, i, j)
    exp_b_3d = to_3d_array(exp_df, s2, i, j)
    control_corr = population_vector_correlation(control_a_3d, control_b_3d, full='off')
    exp_corr = population_vector_correlation(exp_a_3d, exp_b_3d, full='off')

    if control_corr is None or exp_corr is None:
        print(f"Not enough neurons (<5) to compute correlations for {s1} and {s2}. Adjust your data.")
        continue

    control_corr_flat = control_corr.flatten()
    exp_corr_flat = exp_corr.flatten()

    bin_size = 0.05
    bins = np.arange(min(control_corr_flat.min(), exp_corr_flat.min()), 
                     max(control_corr_flat.max(), exp_corr_flat.max()) + bin_size, 
                     bin_size)

    # Compute histograms (counts, not density)
    control_hist, control_bin_edges = np.histogram(control_corr_flat, bins=bins)
    exp_hist, exp_bin_edges = np.histogram(exp_corr_flat, bins=bins)
    
    # Calculate fractions
    control_frac = control_hist / control_hist.sum()
    exp_frac = exp_hist / exp_hist.sum()

    # Calculate bin centers
    control_bin_centers = (control_bin_edges[:-1] + control_bin_edges[1:]) / 2
    exp_bin_centers = (exp_bin_edges[:-1] + exp_bin_edges[1:]) / 2

    # Create pseudo-datasets by multiplying bin counts with bin centers
    control_pseudo = []
    exp_pseudo = []
    
    for count, center in zip(control_hist, control_bin_centers):
        control_pseudo.extend([center] * int(count))
    
    for count, center in zip(exp_hist, exp_bin_centers):
        exp_pseudo.extend([center] * int(count))

    # Convert to numpy arrays
    control_pseudo = np.array(control_pseudo)
    exp_pseudo = np.array(exp_pseudo)

    # Compute CDFs for plotting (using original data)
    control_hist_density, _ = np.histogram(control_corr_flat, bins=bins, density=True)
    exp_hist_density, _ = np.histogram(exp_corr_flat, bins=bins, density=True)
    control_cdf = np.cumsum(control_hist_density) / np.sum(control_hist_density)
    exp_cdf = np.cumsum(exp_hist_density) / np.sum(exp_hist_density)


    # Plot CDFs
    axes[idx].plot(control_bin_edges[:-1], control_cdf, label=f'Control ({s1}, {s2})', 
                      color='blue', linestyle='-', drawstyle='steps-post')
    axes[idx].plot(exp_bin_edges[:-1], exp_cdf, label=f'Exp ({s1}, {s2})', 
                      color='red', linestyle='-', drawstyle='steps-post')
    axes[idx].set_xlabel('Correlation of PV')
    axes[idx].set_ylabel("% Spatial bins")
    axes[idx].legend().set_visible(False)
    axes[idx].spines['top'].set_visible(False)
    axes[idx].spines['right'].set_visible(False)
    axes[idx].set_aspect('auto')
    axes[idx].set_xlim([-0.2, 1])

    # Calculate means and SEM
    control_mean = np.mean(control_corr_flat)
    exp_mean = np.mean(exp_corr_flat)
    control_sem = stats.sem(control_corr_flat)
    exp_sem = stats.sem(exp_corr_flat)

    # Create inset axes for bar plot in top-right corner
    inset_ax = inset_axes(axes[idx], width="100%", height="100%", loc='lower right',
                     bbox_to_anchor=(0.7, 0.05, 0.3, 0.3),  # 4-tuple: x0, y0, width, height
                     bbox_transform=axes[idx].transAxes)
    
    inset_ax.bar([0.2, 0.8], [control_mean, exp_mean], yerr=[control_sem, exp_sem], 
                 color=['blue', 'red'], alpha=0.9, width=0.45, capsize=3, error_kw={'elinewidth': 0.1})
    inset_ax.set_xticks([0.2, 0.8])
    inset_ax.set_xticklabels(['CRs +', 'CRs -'])
    #inset_ax.set_ylabel('Mean Corr')
    inset_ax.tick_params(axis='both')
    inset_ax.spines['top'].set_visible(False)
    inset_ax.spines['right'].set_visible(False)
    inset_ax.set_ylim([0, 0.6])
    inset_ax.set_xticks([])
    inset_ax.set_yticks([])
    #inset_ax.set_xticklabels(['CRs +', 'CRs -'], fontsize=8)
    #inset_ax.set_ylabel('Mean', fontsize=8)
    inset_ax.tick_params(axis='both')
    inset_ax.spines['top'].set_visible(False)
    inset_ax.spines['right'].set_visible(False)
    inset_ax.spines['left'].set_visible(False)

    u_stat, p_value_u = stats.mannwhitneyu(control_corr_flat, exp_corr_flat, alternative='two-sided')
    ks_stat, p_value_ks = stats.ks_2samp(control_pseudo, exp_pseudo)

    y_max = inset_ax.get_ylim()[1]
    bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
    x_positions = [0.2, 0.8]  # Adjusted positions for 'Control' and 'Experimental' groups
    if (p_value_u < 0.05) & (p_value_u  > 0.01):
        inset_ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        inset_ax.text(x_positions[1]*0.5, y_max + bar_height * 1.1, f'*', ha='center', va='bottom')
    elif (p_value_u < 0.01) & (p_value_u  > 0.001):
        inset_ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                            color='black', lw=1.5)
        inset_ax.text(x_positions[1]*0.5, y_max + bar_height * 1.1, f'**', ha='center', va='bottom')
    if (p_value_u < 0.001) :
        inset_ax.plot([x_positions[0], x_positions[1]], [y_max+ bar_height, y_max+ bar_height], 
                        color='black', lw=1.5)
        inset_ax.text(x_positions[1]*0.6, y_max + bar_height * 1.1, f'***', ha='center', va='bottom')
    inset_ax.set_ylim([0, 0.7])
    # Statistical tests
    u_stat, p_value_u = stats.mannwhitneyu(control_corr_flat, exp_corr_flat, alternative='two-sided')
    ks_stat, p_value_ks = stats.ks_2samp(control_pseudo, exp_pseudo)
    print(f"Pattern {s1} vs. {s2}: MW p-value = {p_value_u:.4f}, KS p-value = {p_value_ks:.4f}")


fig.subplots_adjust(top=0.92, bottom=0.08, left=0.1, right=0.95, hspace=0.3, wspace=0.3)




In [ ]:
 y_max

In [ ]:
import pandas as pd
import numpy as np

def process_dataframe(df, normalize=True):
    # Drop rows where ratemaps is NaN/None
    df = df.dropna(subset=['rate_maps'])
    
    # Define control and experimental groups
    control_ids = ['65165', '65091', '63383', '66539', '65622']
    exp_ids = ['65588', '63385', '66538', '66537', '66922']
    # Create new dataframe to store results
    result_df = pd.DataFrame()
    
    # Process each row
    for index, row in df.iterrows():
        animal_id = row['animal_id']
        ratemaps = row['rate_maps']
        
        # Determine group
        if animal_id in control_ids:
            group = 'control'
        elif animal_id in exp_ids:
            group = 'exp'
        
        # Create new row dictionary
        new_row = {'animal_id': animal_id, 'group': group}
        # Handle different ratemaps lengths and optionally normalize
        if len(ratemaps) == 3:
            # Process each map with optional normalization
            if normalize:
                new_row['A'] = ratemaps[0] / np.nanmean(ratemaps[0]) if np.nanmean(ratemaps[0]) != 0 else ratemaps[0]
                new_row['B'] = ratemaps[1] / np.nanmean(ratemaps[1]) if np.nanmean(ratemaps[1]) != 0 else ratemaps[1]
                new_row['C'] = ratemaps[2] / np.nanmean(ratemaps[2]) if np.nanmean(ratemaps[2]) != 0 else ratemaps[2]
            else:
                new_row['A'] = ratemaps[0]
                new_row['B'] = ratemaps[1]
                new_row['C'] = ratemaps[2]
            
        elif len(ratemaps) == 2:
            # Process maps with optional normalization
            if normalize:
                map1 = ratemaps[0] / np.nanmean(ratemaps[0]) if np.nanmean(ratemaps[0]) != 0 else ratemaps[0]
                map2 = ratemaps[1] / np.nanmean(ratemaps[1]) if np.nanmean(ratemaps[1]) != 0 else ratemaps[1]
            else:
                map1 = ratemaps[0]
                map2 = ratemaps[1]

            # Assign maps based on session info
            if 'Session1' in row and 'Session2' in row:
                if row['Session1'] == 1.0:
                    new_row['A'] = map1
                    if row['Session2'] == 2.0:
                        new_row['B'] = map2
                    elif row['Session2'] == 3.0:
                        new_row['C'] = map2
                
                elif row['Session1'] == 2.0:
                    new_row['B'] = map1
                    new_row['C'] = map2
        
        # Add row to result dataframe
        result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)
    return result_df

# Example usage:
pairs_df = pd.read_pickle(r"Q:\sachuriga\CR_CA1_paper\tables/Remapping_ratemaps_unsmoothed.pkl")
# pairs_df = pairs_df[(pairs_df['cell_type1']=="pyramidal") 
#               & (pairs_df['cell_type2']=="pyramidal")]

# Process with normalization (default)
processed_df_normalized = process_dataframe(pairs_df, normalize=True)
print("Normalized DataFrame:")
print(processed_df_normalized.head())

# Process without normalization
processed_df_unnormalized = process_dataframe(pairs_df, normalize=False)
print("\nUnnormalized DataFrame:")
print(processed_df_unnormalized.head())

In [ ]:
import numpy as np
import pandas as pd

def population_vector_correlation(stack1, stack2, full='off', orientation='v'):
    """
    Calculates correlation between population vectors.
    Parameters:
    -----------
    stack1 : ndarray
        3D matrix (neurons, x_bins, y_bins) - population vector
    stack2 : ndarray
        3D matrix (neurons, x_bins, y_bins) - population vector
    full : str, optional
        'off': returns diagonal elements (2D matrix)
        'on': returns full correlation matrices (3D matrix)
        'vector': returns correlations as a vector (1D array)
        Default is 'vector'
    orientation : str, optional
        'v': vertical (rows/x-axis), 'h': horizontal (columns/y-axis)
        Only used when full='vector'. Default is 'v'
    
    Returns:
    --------
    pv_corr : ndarray or None
        Correlation values (1D, 2D, or 3D depending on 'full'), or None if insufficient neurons
    """
    
    # Validate inputs and check neuron count
    if stack1.ndim != 3 or stack2.ndim != 3:
        raise ValueError("Input stacks must be 3D arrays")
    
    num_cells1 = stack1.shape[0]
    num_cells2 = stack2.shape[0]
    
    if num_cells1 < 5 or num_cells2 < 5:
        return None
    
    # Get dimensions
    num_cells = min(num_cells1, num_cells2)
    num_x_bins = min(stack1.shape[1], stack2.shape[1])
    num_y_bins = min(stack1.shape[2], stack2.shape[2])
    
    if num_cells1 != num_cells2:
        print(f"Warning: Population vectors have different neuron counts: {num_cells1} vs {num_cells2}")
    
    # Truncate stacks to minimum dimensions
    stack1 = stack1[:num_cells, :num_x_bins, :num_y_bins]
    stack2 = stack2[:num_cells, :num_x_bins, :num_y_bins]
    
    # Initialize output based on return format
    if full == 'vector':
        num_bins = num_x_bins if orientation == 'v' else num_y_bins
        pv_corr = np.zeros(num_bins)
    elif full == 'off':
        pv_corr = np.zeros((num_x_bins, num_y_bins))
    elif full == 'on':
        pv_corr = np.zeros((num_x_bins, num_y_bins, num_cells))
    else:
        raise ValueError("full must be 'off', 'on', or 'vector'")
    
    # Process based on orientation and return format
    if full == 'vector':
        if orientation == 'v':
            for i in range(num_x_bins):
                vec1 = stack1[:, i, :].reshape(-1)
                vec2 = stack2[:, i, :].reshape(-1)
                pv_corr[i] = np.corrcoef(vec1, vec2)[0, 1]
        else:  # 'h'
            for i in range(num_y_bins):
                vec1 = stack1[:, :, i].reshape(-1)
                vec2 = stack2[:, :, i].reshape(-1)
                pv_corr[i] = np.corrcoef(vec1, vec2)[0, 1]
    
    else:  # full = 'off' or 'on'
        for i in range(num_x_bins):
            for j in range(num_y_bins):
                vec1 = stack1[:, i, j]
                vec2 = stack2[:, i, j]
                corr_matrix = np.corrcoef(vec1, vec2)
                if full == 'off':
                    pv_corr[i, j] = corr_matrix[0, 1]
                else:  # 'on'
                    pv_corr[i, j, :] = corr_matrix[0, :]
    
    # Replace NaN values with 0
    pv_corr = np.nan_to_num(pv_corr, nan=0)
    
    return pv_corr

# Define control and experimental animal IDs
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']

# Initialize dictionaries to store PVs for

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_population_vector_3d_heatmap(rate_maps, position,ax=None):
    """
    Plot stacked 2D heatmaps in 3D and a population vector at the specified position.
    
    Parameters:
    rate_maps : ndarray of shape (n_cells, height, width), e.g., (15, 20, 20)
    position : tuple (x, y), position to extract the population vector, e.g., (18, 18)
    """
    # Extract dimensions
    n_cells, height, width = rate_maps.shape
    x_pos, y_pos = position
    
    
    # Plot each rate map as a 2D heatmap at different Z levels
    for z in range(n_cells + 2):

        if z < n_cells:
            # Create a grid for the heatmap
            X, Y = np.meshgrid(np.arange(width), np.arange(height))
            # Normalize the rate map for colormap
            rate_map = rate_maps[z]
            # Create an RGBA image for the heatmap
            cmap = plt.cm.hot(rate_map / np.max(rate_map))
            # Plot the heatmap as a collection of colored squares
            for i in range(height):
                for j in range(width):
                    color = cmap[i, j]
                    ax.bar3d(j, i, z, 1, 1, 0, color=color, shade=False, alpha=0.6)

        if z == n_cells+1:
            X, Y = np.meshgrid(np.arange(width), np.arange(height))
            # Normalize the rate map for colormap
            rate_map = rate_maps[3]
            # Create an RGBA image for the heatmap
            cmap = plt.cm.hot(rate_map / np.max(rate_map))
            # Plot the heatmap as a collection of colored squares
            for i in range(height):
                for j in range(width):
                    color = cmap[i, j]
                    ax.bar3d(j, i, -3, 1, 1, 0, color=color, shade=False, alpha=0.6)

    # Add a vertical line at (x=15, y=15)
    ax.plot3D([x_pos, x_pos], [y_pos, y_pos],[-4, 8], color='blue')
    
 
    # Set labels and limits
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Cell Index')
    ax.set_zlim(0, n_cells-1)
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    
    # Customize z-axis ticks to show "1, 2, 3, ..., n"
    z_ticks = np.arange(n_cells)
    z_labels = [str(i+1) for i in range(n_cells)]
    z_labels[-1] = 'n'  # Replace the last label with 'n'
    ax.set_zticks(z_ticks)
    ax.set_zticklabels(z_labels)
    
    # Adjust the viewing angle for a more parallel perspective
    ax.view_init(elev=10, azim=45)  # Elevation = 10°, Azimuth = 45°
    plt.axis('off')
    # Save the plot
    plt.savefig('population_vector_3d_heatmap_plot.png')
    plt.show()